# Operations

Operations is a type of a `Expression`, some operations would be `Add`, `And` or `Greater`.

Operations should have 2 steps, where the last one should return the result type. 
For example, the `Add` receives two `NumericValue` parameters `left` and `right` and returns a 
`NumericValue` expression:

```
n1 = Numeric(value=1)
n2 = Numeric(value=2)

(n1 + m2) -> Add(n1, n2) -> NumericScalar(source=Add(n1, n2))
```

Where `NumericValue` could be a `NumericScalar` or a `NumericColumn`.

In [1]:
from __future__ import annotations
import typing
from typing import NewType, Union, Optional, List, Callable, NewType
import dataclasses

import metadsl
import metadsl_rewrite
import metadsl_core
import pandas as pd

from toki import datatypes as dts
from toki import operations as ops
from toki import rules as rls
from toki import types as tps
from toki.backends.core import Backend, BackendTranslator
from toki.viz import visualize

In [2]:
sch = tps.TableSchema.expr({
    'col_1': {'type': 'int64', 'nullable': True},
    'col_2': {'type': 'float64', 'nullable': True},
})

print(sch)

TableSchema
  col_1: int64(nullale)
  col_2: float64(nullale)



In [3]:
dts.Number.expr(1) + dts.Number.expr(1)

Add([Number([1]), Number([1])])

In [4]:
dts.int8(1) + dts.int8(8)

Add([int8([1]), int8([8])])

In [5]:
dts.int8(1) - dts.int8(8)

Subtract([int8([1]), int8([8])])

In [6]:
table = tps.Table.expr(
    'my_table', 
    sch, 
    database_name='postgres', 
    database_schema_name='public'
)
table

postgres.public.my_table: Table
  col_1: int64(nullale)
  col_2: float64(nullale)

In [7]:
type(table)

toki.types.Table

In [8]:
table['col_1'] + 1

Add([IntegerColumn[col_1]
  postgres.public.my_table: Table
    col_1: int64(nullale)
    col_2: float64(nullale)
  
, 1])

In [9]:
table[['col_1']]

Projection[['col_1']]
  postgres.public.my_table: Table
    col_1: int64(nullale)
    col_2: float64(nullale)
  

In [10]:
table['col_1'].name('tmp')

IntegerColumn[col_1](as tmp)
  postgres.public.my_table: Table
    col_1: int64(nullale)
    col_2: float64(nullale)
  

In [11]:
table['col_1']

IntegerColumn[col_1]
  postgres.public.my_table: Table
    col_1: int64(nullale)
    col_2: float64(nullale)
  

In [12]:
expr_comp = (table['col_1'] > table['col_1'])
expr_comp

GreaterThan([IntegerColumn[col_1]
  postgres.public.my_table: Table
    col_1: int64(nullale)
    col_2: float64(nullale)
  
, IntegerColumn[col_1]
  postgres.public.my_table: Table
    col_1: int64(nullale)
    col_2: float64(nullale)
  
])

In [13]:
isinstance(expr_comp, ops.LogicalBinaryOp)

False

In [14]:
(expr_comp and expr_comp)

GreaterThan([IntegerColumn[col_1]
  postgres.public.my_table: Table
    col_1: int64(nullale)
    col_2: float64(nullale)
  
, IntegerColumn[col_1]
  postgres.public.my_table: Table
    col_1: int64(nullale)
    col_2: float64(nullale)
  
])

In [15]:
class NewType(dts.Number):
    """New type."""


class OperationExpr(tps.Expr):
    """Operation expression."""

    result: Type = dts.DataType

    @property
    def _display_name(self) -> str:
        return '{}[{}]'.format(super()._display_name(), self.result.__name__)
    
    
class BinaryOp(OperationExpr):
    """Binary base operation."""

    @property
    def left(self) -> dts.DataType:
        return self.args[0]

    @property
    def right(self) -> dts.DataType:
        return self.args[1]
    
    @staticmethod
    @tps.constructor
    def inner_expr(left: dts.DataType, right: dts.DataType) -> BinaryOp:
        """Create an BinaryOp expression for the given parameters."""


    @classmethod
    def expr(cls, left: dts.DataType, right: dts.DataType) -> dts.DataType:
        """Create an BinaryOp expression for the given parameters."""
        DataT = cls.result
        
        v = cls.inner_expr(left, right)
            
        return DataT.expr(v)
            
class Add(BinaryOp):
    result = dts.Number
    

expr = Add.expr(NewType.expr(1), NewType.expr(1))
expr

TypeError: Cannot match concrete type <class '__main__.BinaryOp'> with hint typing.Union[int, float]